In [46]:
#//----------------------------
#//LIBRARIES
    #Math
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time
    #json library
import json
    #Analyst tools
import sys
sys.path.append('../')
from analysts_tools.growth import *
    #Procurement tools
from procurement_lib import send_slack_notification
from procurement_lib import redash
from analysts_tools.redash_methods import *

import random
import time

In [47]:
#//---------------------------------------------------------------------------------------------------------------------------
#// Parametros
"""Todays date should be the first day of the next month which is going to begin with SD"""
city_code = 'BOG'
    #year-month-day
todays_date = '2022-09-20' #When the offer ends
customer_date_desde = '2022-09-14' #When the offer is available 


tasas = {'BOG': 3776,
         'BAQ': 3776,
         'MDE': 3776,
         'CMX': 19.65,
         'GDL': 19.65,
         'PBC': 19.65,
         'SPO': 4.75,
         'BHZ': 4.75,
         'CWB': 4.75,
         'VCP': 4.75}

t_cambio = tasas[city_code]

In [48]:
query = """
SELECT DISTINCT
    bo.submit_date,
    TO_CHAR(bo.submit_date, 'YYYY-mm') AS month,
    bo.order_id,
    bo.customer_id,
    boi.name,
    (boi.quantity * foi.step_unit * boi.sale_price)/{tipo_cambio} as gmv_usd,
    COALESCE((baid.adjustment_value * boip.quantity * foi.step_unit)/{tipo_cambio}, 0) as discount_applied,
    COALESCE(foi.total_tax_iva/{tipo_cambio},0) as taxes,
    boi.quantity* foi.step_unit as cant,
    UPPER(fpro.unit) as unidades,
    bs.sku_id,
    bs.addl_product_id as padre_sku_id,
    s.site_identifier_value as region_code,
    CASE
        WHEN COALESCE(bcat2.name, bcat.name) ILIKE 'Proteínas%' THEN 'Proteínas'
        ELSE COALESCE(bcat2.name, bcat.name) 
    END AS category,
    CASE
        WHEN bcat.name ILIKE 'Pollo%' THEN 'Pollo'
        WHEN bcat.name ILIKE 'Res%' THEN 'Res'
        WHEN bcat.name ILIKE 'Cerdo%' THEN 'Cerdo'
        ELSE bcat.name
    END AS subcat,
    baid.adjustment_reason
FROM postgres_broadleaf_federate."broadleaf.blc_fulfillment_group_item"     bfgi
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_group"    bfg        ON bfgi.fulfillment_group_id = bfg.fulfillment_group_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_fulfillment_group"     ffg        ON ffg.fulfillment_group_id = bfg.fulfillment_group_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order"                bo         ON bo.order_id = bfg.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site"                 s          ON bo.site_disc = s.site_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_item"           boi        ON bfgi.order_item_id= boi.order_item_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order_item"            foi        ON boi.order_item_id= foi.order_item_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_owner"                 fow        ON fow.owner_id = ffg.owner_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order"                 fo         ON fo.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_order"    bfo        ON bfo.fulfillment_group_id= bfg.fulfillment_group_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_discrete_order_item"  bdoi       ON bdoi.order_item_id = boi.order_item_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                  bs         ON bs.sku_id = bdoi.sku_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_product"              bp         ON bs.addl_product_id = bp.product_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category"             bcat       ON bcat.category_id = bp.default_category_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_product"               fpro       ON fpro.product_id = bp.product_id
LEFT JOIN  postgres_broadleaf_federate."broadleaf.blc_order_payment"        bop        ON bop.order_id = bo.order_id
LEFT JOIN  postgres_broadleaf_federate."broadleaf.fb_order_type"            fot        ON fot.fb_order_type_id=fo.fb_order_type_id
LEFT JOIN  postgres_broadleaf_federate."broadleaf.fb_customer"              fc         ON fc.customer_id = bo.customer_id
LEFT JOIN  postgres_broadleaf_federate."broadleaf.fb_business_type"         bt         ON fc.business_type_id = bt.business_type_id
LEFT JOIN  postgres_broadleaf_federate."broadleaf.blc_category_xref"        bcx        ON bcx.sub_category_id = bp.default_category_id AND bcx.archived='N' AND bcx.sndbx_tier is NULL
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_category"             bcat2      ON bcx.category_id = bcat2.category_id
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_price_dtl"  boip       ON  boi.order_item_id = boip.order_item_id
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_dtl_adj"    baid       ON  baid.order_item_price_dtl_id = boip.order_item_price_dtl_id

WHERE 
   DATE(bo.submit_date) between '{desde}' AND '{hasta}'
   AND bo.order_status='SUBMITTED'
   AND fo.fb_order_status_id IN (1,6,7,8) 
   AND s.site_identifier_value IN ('{ciudad}')
   AND bfo.status NOT IN ('ARCHIVED','CANCELLED')
   AND bop.archived = 'N'
   AND (fot.name IS NULL OR fot.name <> 'REFUND')
   AND fc.business_type_id IN (1)
   -- FIX SUPER DESCUENTOS
   AND bcat2.category_id not in ('110873','-1000','100768','100765','100815') --ids de super descuentos en cada país
   AND bcat.name <> 'Oferton Frubana' AND bcat2.name <> 'Oferton Frubana'
""".format(tipo_cambio=t_cambio, desde=customer_date_desde, hasta=todays_date, ciudad=city_code)

dataframe = read_connection_data_warehouse.runQuery(query)
dataframe[["gmv_usd", "cant"]] = dataframe[["gmv_usd", "cant"]].astype(float)
dataframe["submit_date"] = pd.to_datetime(dataframe["submit_date"])
dataframe["submit_date"] = dataframe["submit_date"].dt.strftime("%d-%m-%y")

In [49]:
dataframe.columns

Index(['submit_date', 'month', 'order_id', 'customer_id', 'name', 'gmv_usd',
       'discount_applied', 'taxes', 'cant', 'unidades', 'sku_id',
       'padre_sku_id', 'region_code', 'category', 'subcat',
       'adjustment_reason'],
      dtype='object')

In [50]:
def get_info(dff):
    df = dff.groupby(by=['padre_sku_id']).apply(#the change will be here padre_sku_id // padre_sku_id
        lambda d: pd.Series(
            {
                "name": d['name'].unique(),
                "gmv_usd": d["gmv_usd"].sum(),
                "cant": d["cant"].sum(),
                "customer_id": d["customer_id"].nunique(),
                "discount_usd": d["discount_applied"].sum(),
                "avg_gmv": d["gmv_usd"].sum() / d["customer_id"].nunique(),
                "avg_cant": d["cant"].sum() / d["customer_id"].nunique(),
                "avg_discount": float(d["discount_applied"].sum() / d["customer_id"].nunique()),
                "% dct": float(d["discount_applied"].sum()) *100.0 / d["gmv_usd"].sum()

            }
        )
    ).reset_index()
    return df

In [51]:
def delta(control_results,test_results,control,piloto):
    delta_df = pd.DataFrame()
    delta_df['padre_sku_id'] = test_results['padre_sku_id']
    delta_df['name'] = test_results['name']
    #delta_df['total_control_customers'] = control
    #delta_df['total_test_customers'] = piloto
    delta_df['control_customers'] = control_results['customer_id']
    delta_df['test_customers'] = test_results['customer_id']
    delta_df['delta_avg_gmv'] = test_results['avg_gmv'] - control_results['avg_gmv']
    delta_df['delta_avg_cant'] = test_results['avg_cant'] - control_results['avg_cant']
    delta_df['delta_avg_discount'] = test_results['avg_discount'] - control_results['avg_discount']
    delta_df['delta_conversion'] = ((delta_df['test_customers']/piloto) - (delta_df['control_customers']/control))*100.00
    delta_df['delta_% dct'] = test_results['% dct'] - control_results['% dct']
    return  delta_df

In [52]:
#Colocar CSVs de los experimentos
#SD/export_data/Below_Media/MDE/2022-08-26_csv_MDE_COCACOLA_All_CLIENTS.csv
exp_sin_restriccion = pd.read_csv(f"SD/export_data/Below_Media/{city_code}/{customer_date_desde}_OFFER_csv_{city_code}_All_SH.csv")#['costumer_id'].unique().tolist()
clients_exp_sin_restriccion = pd.read_csv(f"SD/export_data/Below_Media/{city_code}/{customer_date_desde}_CLIENTS_csv_{city_code}_All_SH.csv")#['costumer_id'].unique().tolist()

In [53]:
exp_sin_restriccion

,offer_name,offer_description,discount,automatically_consider_offer,start_date,end_date,max_uses_per_order,max_uses_per_customer,customer_segment_id,sku_id
0,SH_2022-09-14_BOG_CocaCola,NaN,11,True,14-09-2022,19-09-2022,50,3,82955,227091#194399


In [54]:
delta_df = []
control_df = []
test_df = []

for offer in exp_sin_restriccion.values:
    
    control = list(map(int,clients_exp_sin_restriccion['CONTROL_'+offer[0]].dropna().unique()))
    piloto = list(map(int,clients_exp_sin_restriccion['TEST_'+offer[0]].dropna().unique()))
    
    if isinstance(offer[-1], str):
        skus = list(map(int,offer[-1].split("#")))
    else:
        skus = [offer[-1]]
    
    df_test = dataframe[(dataframe.sku_id.isin(skus)) & (dataframe.customer_id.isin(piloto)) & (dataframe.submit_date >= offer[4]) & (dataframe.submit_date <= offer[5])] 
    df_control = dataframe[(dataframe.sku_id.isin(skus)) & (dataframe.customer_id.isin(control)) & (dataframe.submit_date >= offer[4]) & (dataframe.submit_date <= offer[5])]
    
    print(f'Test: {offer[0]} -> control users: {len(control)} / piloto users: {len(piloto)}')
    
    if df_control.empty and df_test.empty:
        print('DataFrame is empty!')
        control_results = pd.DataFrame(columns = ['sku_id', 'name', 'gmv_usd', 'cant', 'customer_id', 'discount_usd',
       'avg_gmv', 'avg_cant', 'avg_discount', '% dct'])
        test_results = pd.DataFrame(columns = ['sku_id', 'name', 'gmv_usd', 'cant', 'customer_id', 'discount_usd',
       'avg_gmv', 'avg_cant', 'avg_discount', '% dct']) 
    elif df_control.empty and (not df_test.empty):     
        print('Control DataFrame is empty!')
        test_results = get_info(df_test)
        control_results = pd.DataFrame(columns = ['sku_id', 'name', 'gmv_usd', 'cant', 'customer_id', 'discount_usd',
       'avg_gmv', 'avg_cant', 'avg_discount', '% dct'])
    elif df_test.empty and (not df_control.empty):
        print('Test DataFrame is empty!')
        control_results = get_info(df_control)
        test_results = pd.DataFrame(columns = ['sku_id', 'name', 'gmv_usd', 'cant', 'customer_id', 'discount_usd',
       'avg_gmv', 'avg_cant', 'avg_discount', '% dct'])   
    else:
        control_results = get_info(df_control)
        test_results = get_info(df_test)
        
    #------------------------#-------------------------#
    control_df.append(control_results)
    test_df.append(test_results)
    delta_df.append(delta(control_results,test_results,len(control),len(piloto)))

Test: SH_2022-09-14_BOG_CocaCola -> control users: 263 / piloto users: 1488


In [56]:
test_df[0]

,padre_sku_id,name,gmv_usd,cant,customer_id,discount_usd,avg_gmv,avg_cant,avg_discount,% dct
0,157414,[Coca Cola 400 ml x 12 un Paca x 12 un - Desd...,624.908891,117.0,76,7.0502538,8.222485,1.539474,0.092766,1.128205
1,184310,[Coca Cola 1.5lt pet caja x 12 un Paca x 12 U...,1828.423195,158.0,114,171.9643534,16.038800,1.385965,1.508459,9.405063


In [57]:
control_df[0]

,padre_sku_id,name,gmv_usd,cant,customer_id,discount_usd,avg_gmv,avg_cant,avg_discount,% dct
0,157414,[Coca Cola 400 ml x 12 un Paca x 12 un - Desd...,53.411016,10.0,9,0E-7,5.934557,1.111111,0.000000,0.000000
1,184310,[Coca Cola 1.5lt pet caja x 12 un Paca x 12 U...,439.747351,38.0,21,42.1231664,20.940350,1.809524,2.005865,9.578947


In [58]:
delta_df[0]

,padre_sku_id,name,control_customers,test_customers,delta_avg_gmv,delta_avg_cant,delta_avg_discount,delta_conversion,delta_% dct
0,157414,[Coca Cola 400 ml x 12 un Paca x 12 un - Desd...,9,76,2.287928,0.428363,0.092766,1.685474,1.128205
1,184310,[Coca Cola 1.5lt pet caja x 12 un Paca x 12 U...,21,114,-4.901550,-0.423559,-0.497406,-0.323501,-0.173884
